In [2]:
strides = [2,2,2,2,3]
print(strides[::-1])

[3, 2, 2, 2, 2]


In [27]:
import torch
from monai.transforms import AsDiscrete
ds = "/Users/keyi/Desktop/MA/AutoPET_111/FDG-PET-CT-Lesions/sample_1_train.pt"
ds = torch.load(ds)

combined_gt = ds[0]['INTERPOLATED_ANA']
combined_gt_2 = ds[1]['INTERPOLATED_ANA']
print(combined_gt_2.shape)
batch = torch.cat([combined_gt, combined_gt_2], dim=0)
batch = batch.unsqueeze(1)
print(batch.shape)
print(batch.unique())
print("Unique values in batch:", batch.unique())
print("Min value:", batch.min())
print("Max value:", batch.max())
combined_gt = AsDiscrete(to_onehot=54)(batch)
print(combined_gt.shape)
print(combined_gt[53])


torch.Size([1, 96, 96, 96])
torch.Size([2, 1, 96, 96, 96])
metatensor([ 0,  1,  2,  3,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 22, 23, 24,
        25, 26, 27, 28, 29, 30, 31, 32, 37, 39, 41, 42, 47, 49, 51, 52, 53],
       dtype=torch.uint8)
Unique values in batch: metatensor([ 0,  1,  2,  3,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 22, 23, 24,
        25, 26, 27, 28, 29, 30, 31, 32, 37, 39, 41, 42, 47, 49, 51, 52, 53],
       dtype=torch.uint8)
Min value: metatensor(0, dtype=torch.uint8)
Max value: metatensor(53, dtype=torch.uint8)


/var/folders/jw/3gmq54sn6q3cnlvsjqly5twr0000gn/T/ipykernel_38850/1122214994.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ds = torch.load(ds)


AssertionError: labels should have a channel with length equal to one.

In [46]:
import torch
from monai.transforms import AsDiscrete
ds = "/Users/keyi/Desktop/MA/AutoPET_111/FDG-PET-CT-Lesions/sample_16_train.pt"
ds = torch.load(ds)

combined_gt = ds[0]['INTERPOLATED_ANA']
combined_gt_2 = ds[2]['INTERPOLATED_ANA']
print(combined_gt_2.shape)
batch = torch.cat([combined_gt, combined_gt_2], dim=0)
batch = batch.unsqueeze(1)
print(batch.shape)
print(batch.unique())
print("Unique values in batch:", batch.unique())
print("Min value:", batch.min())
print("Max value:", batch.max())



combined_gt = AsDiscrete(to_onehot=55, dim=1)(batch)
print(combined_gt.shape)
background = combined_gt[:, 0:1] 
anatomy_gt = combined_gt[:, :54]   
tumor = combined_gt[:, 54:55]     
pathology_gt = torch.cat([background, tumor], dim=1)
print(pathology_gt.shape)

print(anatomy_gt.shape)

torch.Size([1, 96, 96, 96])
torch.Size([2, 1, 96, 96, 96])
metatensor([ 0,  1,  2,  5,  6,  8, 10, 11, 12, 13, 23, 24, 25, 26, 27, 28, 29, 30,
        31, 39, 40, 41, 42, 47, 49, 51, 52, 53, 54], dtype=torch.uint8)
Unique values in batch: metatensor([ 0,  1,  2,  5,  6,  8, 10, 11, 12, 13, 23, 24, 25, 26, 27, 28, 29, 30,
        31, 39, 40, 41, 42, 47, 49, 51, 52, 53, 54], dtype=torch.uint8)
Min value: metatensor(0, dtype=torch.uint8)
Max value: metatensor(54, dtype=torch.uint8)
torch.Size([2, 55, 96, 96, 96])
torch.Size([2, 2, 96, 96, 96])
torch.Size([2, 54, 96, 96, 96])


/var/folders/jw/3gmq54sn6q3cnlvsjqly5twr0000gn/T/ipykernel_38850/317556288.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ds = torch.load(ds)


In [1]:
import torch.nn.functional as F
import torch
feat = torch.randn(1,1024,3,3,3)
feat_2 = torch.clone(feat)

def compute_readiness_loss(path_feat, anat_feat):
        path_stats = torch.cat([
            path_feat.mean(dim=[2,3,4]),
            path_feat.std(dim=[2,3,4])
        ], dim=1)
        
        anat_stats = torch.cat([
            anat_feat.mean(dim=[2,3,4]),
            anat_feat.std(dim=[2,3,4])
        ], dim=1)
        
        readiness_loss = F.mse_loss(path_stats, anat_stats)
        readiness_score = -readiness_loss.item()
        
        return readiness_loss, readiness_score

compute_readiness_loss(feat, feat_2)

(tensor(0.), -0.0)